Process the Customers Data 
- 1.Ingest the data into the data lakehouse - bronze_customers
- 2.Perform the data quality and transform the data as required -- silver_customers_clean
- 3.Apply Changes to the customers data - silver_customers

 1.Ingest the data into the data lakehouse - bronze_customers

In [0]:
CREATE OR REFRESH STREAMING TABLE circuitbox.bronze.customers_fi
comment 'Raw comments data ingested from the source system operational_data'
TBLPROPERTIES('qulaity'='bronze')
as 
select *,
      _metadata.file_path as input_file_path,
      current_timestamp() as ingestion_timestamp 
from cloud_files(
  '/Volumes/circuitbox/landing/operational_data/customers/',
  'json',
  map('cloudFiles.inferColumnTypes','true')

);

2.Perform data quality checks and transform the data as required -- silver_customers_clean

In [0]:
create or refresh streaming table circuitbox.silver.customers_clean(
  constraint valid_customer_id expect(customer_id is not null) on violation fail update,
  constraint valid_customer_name expect(customer_name is not null) on violation drop row,
  constraint valid_telephone expect(len(telephone)<10) ,
  constraint valid_email expect(email is not null),
  constraint valid_date_of_birth expect(date_of_birth >='1920-01-01')
)
comment 'Cleaned Customers data'
tblproperties('quality'='silver')
as
select customer_id,
      customer_name,
      cast(date_of_birth as date) as date_of_birth,
      telephone,
      email,
      cast(created_date as date) as created_date
FROM STREAM(circuitbox.bronze.customers_fi)

-- Now lets practtise the CDC uisng SCD TYPE | SCD TYPE2 , this can be achieved by usng the APPLY CHANGES API.
- Note : Target table must be streaming table 

3, Apply SCD Type 1 to the customers data - silver_customers

In [0]:
create or refresh streaming table circuitbox.silver.customers
comment 'SCD TYPE1 CUSTOMERS DATA'
tblproperties("quality"="silver")

In [0]:
CREATE FLOW customers_cdc_flow AS AUTO CDC INTO circuitbox.silver.customers
FROM STREAM(circuitbox.silver.customers_clean)
KEYS (customer_id)
SEQUENCE BY created_date
STORED AS SCD TYPE 2;